In [1]:
import pandas as pd
import os
from tqdm import tqdm
titlePath = '../dataset/Data_TitleInfo2.csv'
studentPath = '../dataset/Data_StudentInfo.csv'
submitPath = '../dataset/Data_SubmitRecord'


# 获取题目数据
def getTitle():
    title = pd.read_csv(titlePath)
    return title

# 获取学生数据
def getStudent():
    student = pd.read_csv(studentPath)
    return student.drop(columns=['index'])

# 获取提交数据（班级号）
def getSubmit(classId=1):
    submit = pd.read_csv(os.path.join(submitPath, 'SubmitRecord-Class' + str(classId) + '.csv'))
    return submit.drop(columns=['index'])

# 所有提交数据
def getAllSubmit():
    submitRecords = []
    for i in range(1, 16):
        submitRecords.append(getSubmit(i))
    return pd.concat(submitRecords)

数据清洗

In [14]:
# 简单清洗并合并数据
def cleanMerge():
    title = getTitle()
    student = getStudent()
    submit = getAllSubmit()

    print(title.shape, student.shape, submit.shape)
    submit = submit.drop_duplicates()
    print(submit.shape)

    # 去除提交记录中的错误student_id
    # print(student['student_ID'].unique().shape)
    # print(submit['student_ID'].unique().shape)
    error_students = submit[~submit['student_ID'].isin(student['student_ID'])]
    # print('errorid',error_students.shape,error_students['student_ID'].iloc[0])
    submit = submit[submit['student_ID'] != error_students['student_ID'].iloc[0]]
    print(submit.shape)


    # 提交记录中，class错误的修正
    print('-----')
    print(submit['class'].unique())
    error_rows = submit[submit['class'] == 'class']
    for index,row in error_rows.iterrows():
        id = row['student_ID']
        # print(id)
        # print(id, sbms.shape, sbms['class'].unique()[0])
        submit.loc[index, 'class'] = submit[submit['student_ID'] == id]['class'].unique()[0]

    print(submit['class'].unique())

    # 检查题目id
    print('-----')
    print(title['title_ID'].unique().shape)
    print(submit['title_ID'].unique().shape)

    print('-----')
    # merge的时候发现，提交数据和题目数据中的score冲突，更改了题目数据中score->fullscore
    # 注意：题目数据使用的是TitleInfo2.csv
    print(submit.shape, submit.columns)
    merge1 = pd.merge(submit, student, on='student_ID')
    print(merge1.shape, merge1.columns)
    merge2 = pd.merge(merge1, title, on='title_ID')
    print(merge2.shape, merge2.columns)

    return merge2

In [19]:
'''
计算人-题数据特征
'''
df = cleanMerge()
students = df['student_ID'].unique()
titles = df['title_ID'].unique()
#print(df[df.duplicated(subset=['student_ID', 'title_ID','time'],keep=False)].sort_values(by=['student_ID',

newDf = pd.DataFrame(columns=['student_ID', 'title_ID', 'st_max_score', 'st_pfm_tc','st_pfm_mem','st_tt_subNum',
                                'st_tt_period','st_fm_subNum','st_fm_period','st_errs','st_exploreNum'])
for student in tqdm(students):
    for title in titles:
        studentTitle = df[(df['student_ID'] == student) & (df['title_ID'] == title)]
        if studentTitle.empty: continue
        max_score = studentTitle['score'].max()
        tt_subNum = studentTitle.shape[0]
        tt_period = studentTitle['time'].max() - studentTitle['time'].min()
        fm_time= studentTitle[studentTitle['score'] == max_score]['time'].min()
        pfm_tc= studentTitle[studentTitle['score'] == max_score]['timeconsume'].min()
        pfm_mem= studentTitle[studentTitle['score'] == max_score]['memory'].min()
        fm_subNum = studentTitle[studentTitle['time'] <= fm_time].shape[0]
        fm_period = fm_time - studentTitle['time'].min()
        st_errs = list(studentTitle[(studentTitle['state'] != 'Absolutely_Correct') & (studentTitle['state'] != 'Partially_Correct')]['state'].unique())
        ac_flag = 1 if (studentTitle[studentTitle['state'] == 'Absolutely_Correct'].shape[0]) > 0 else 0
        ac_time= studentTitle[studentTitle['state'] == 'Absolutely_Correct']['time'].min()
        st_exploreNum = studentTitle[studentTitle['time'] < ac_time].shape[0]

        newDf = pd.concat([newDf,pd.DataFrame([{
            'student_ID':student,'title_ID':title,'st_max_score':max_score,
            'st_pfm_tc':pfm_tc,'st_pfm_mem':pfm_mem,
            'st_fm_time':fm_time,
            'st_tt_subNum':tt_subNum,'st_tt_period':tt_period,
            'st_fm_subNum':fm_subNum,'st_fm_period':fm_period,
            'st_ac_flag':ac_flag,'st_ac_time':ac_time,
            'st_errs':';'.join(st_errs),
            'st_exploreNum':st_exploreNum
            }])],ignore_index=False)
newDf.to_csv('../dataset/Data_StudentTitleInfo.csv',index=False)

(38, 7) (1364, 4) (232818, 9)
(231812, 9)
(231811, 9)
-----
['Class1' 'class' 'Class2' 'Class3' 'Class4' 'Class5' 'Class6' 'Class7'
 'Class8' 'Class9' 'Class10' 'Class11' 'Class12' 'Class13' 'Class14'
 'Class15']
['Class1' 'Class15' 'Class7' 'Class2' 'Class3' 'Class4' 'Class5' 'Class6'
 'Class8' 'Class9' 'Class10' 'Class11' 'Class12' 'Class13' 'Class14']
-----
(38,)
(38,)
-----
(231811, 9) Index(['class', 'time', 'state', 'score', 'title_ID', 'method', 'memory',
       'timeconsume', 'student_ID'],
      dtype='object')
(231811, 12) Index(['class', 'time', 'state', 'score', 'title_ID', 'method', 'memory',
       'timeconsume', 'student_ID', 'sex', 'age', 'major'],
      dtype='object')
(231811, 18) Index(['class', 'time', 'state', 'score', 'title_ID', 'method', 'memory',
       'timeconsume', 'student_ID', 'sex', 'age', 'major', 'fullscore',
       'knowledge_num', 'knowledge', 'sub_knowledge', 'knowledge2',
       'sub_knowledge2'],
      dtype='object')


  0%|          | 0/30 [00:00<?, ?it/s]/var/folders/_n/smz6g_y16tg_x85vpk78z5jh0000gp/T/ipykernel_3387/2168936426.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  newDf = pd.concat([newDf,pd.DataFrame([{
100%|██████████| 30/30 [01:16<00:00,  2.55s/it]
